In [1]:
import ee
import os
import time
import subprocess
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta

# Geemap
try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

try:
    import ipygee
except ImportError:
    print('Installing ipygee ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'ipygee'])

import pprint
import geemap

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

Installing ipygee ...


In [2]:
# Initialize Earth Engine
ee.Authenticate()

ee.Initialize(project='ee-chriscandido93')
print("GEE initialized successfully")

GEE initialized successfully


In [3]:
# =============================
# 1. Define AOI and Time Range
# =============================

min_lon, min_lat, max_lon, max_lat = 119, 6, 125, 16
aoi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

start_date = '2024-01-01'
end_date = '2024-05-31'

print(f"AOI Bounds: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")
print(f"Time Range: {start_date} to {end_date}\n")

AOI Bounds: (119, 6) to (125, 16)
Time Range: 2024-01-01 to 2024-05-31



In [4]:
# Cloud Masking
def mask_clouds_and_quality(image):
    """
    Advanced cloud and quality masking for Sentinel-2
    """
    # QA60 cloud masking
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    qa_mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
               .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))

    # Additional spectral-based cloud/haze removal
    blue = image.select('B2')
    nir = image.select('B8')
    swir1 = image.select('B11')

    # Remove bright pixels (clouds/haze)
    not_bright = blue.lt(2000)  # Raw DN values

    # Remove pixels with cloud-like spectral signature
    cloud_score = blue.add(nir).divide(2).subtract(swir1)
    not_cloud = cloud_score.lt(500)

    # Shadow removal (very dark pixels)
    not_shadow = nir.gt(300)

    # Combine all masks
    combined_mask = (qa_mask
                     .And(not_bright))

    # Scale to reflectance
    return (image.updateMask(combined_mask)
            .divide(10000)
            .copyProperties(image, ["system:time_start", "CLOUDY_PIXEL_PERCENTAGE"])
    )

def add_cloud_score(image):
    """
    Add a custom cloud score to each image
    """
    # Calculate cloud probability
    blue = image.select('B2')
    nir = image.select('B8')
    swir1 = image.select('B11')

    cloud_score = (blue.add(nir).divide(2).subtract(swir1)
                   .divide(blue.add(nir).divide(2))
                   .multiply(100)
                   .rename('cloud_score'))

    return image.addBands(cloud_score)

def create_composite(aoi, start_date, end_date, max_cloud_cover=20):
    """
    Create improved Sentinel-2 composite with multiple quality filters

    Args:
        aoi: Area of interest
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        max_cloud_cover: Maximum cloud cover percentage (default: 20)
    """

    print(f"\n{'='*60}")
    print("Creating Improved Sentinel-2 Composite")
    print(f"{'='*60}")

    # Load Sentinel-2 Surface Reflectance
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))
          .map(mask_to_reef)
    )

    print(f"Initial images: {s2.size().getInfo()}")

    # Apply cloud masking
    s2_masked = s2.map(mask_clouds_and_quality)

    # Add cloud score
    s2_scored = s2_masked.map(add_cloud_score)

    # Percentile composite (reduces outliers)
    composite_percentile = (s2_masked.reduce(ee.Reducer.percentile([50]))
                           .select(['B1_p50','B2_p50','B3_p50','B4_p50',
                                   'B8_p50','B8A_p50','B11_p50','B12_p50'],
                                  ['B1','B2','B3','B4','B8','B8A','B11','B12'])
                           .clip(aoi))
    return {'percentile': composite_percentile}

def enhance_water_clarity(composite):
    """
    Enhance water clarity for reef/coastal imaging
    Apply atmospheric correction and water enhancement
    """

    # Sun glint removal using SWIR
    nir = composite.select('B8')
    swir1 = composite.select('B11')

    # Estimate glint in visible bands
    blue = composite.select('B2')
    green = composite.select('B3')
    red = composite.select('B4')

    # Simple deglinting (for better water penetration)
    # Assumes linear relationship between NIR and visible glint
    slope = 0.9  # Adjust based on your area

    blue_deglint = blue.subtract(nir.multiply(slope))
    green_deglint = green.subtract(nir.multiply(slope))
    red_deglint = red.subtract(nir.multiply(slope))

    # Clip to valid range
    blue_deglint = blue_deglint.clamp(0, 1)
    green_deglint = green_deglint.clamp(0, 1)
    red_deglint = red_deglint.clamp(0, 1)

    # Replace bands
    enhanced = composite.addBands(blue_deglint, overwrite=True) \
                        .addBands(green_deglint, ['B3'], True) \
                        .addBands(red_deglint, ['B4'], True)

    return enhanced

def mask_to_reef(image):

    aca = ee.Image("ACA/reef_habitat/v2_0").select("reef_mask")
    reef_mask = aca.eq(1)

    return image.updateMask(reef_mask).copyProperties(image, ["system:time_start"])

In [5]:
# =============================
# 3. Load Satellite (Sentinel2)
# =============================

def create_best_composite(aoi, start_date, end_date, max_cloud_cover=20):
    """
    Create the best possible composite for coastal/reef areas
    Combines multiple techniques
    """

    print(f"\n{'='*70}")
    print("CREATING OPTIMIZED COMPOSITE FOR COASTAL/REEF AREAS")
    print(f"{'='*70}")
    print(f"Date range: {start_date} to {end_date}")
    print(f"Max cloud cover: {max_cloud_cover}%")

    # Step 1: Get composites
    composites = create_composite(aoi, start_date, end_date, max_cloud_cover)

    # Step 2: Use quality mosaic as base (best pixel selection)
    base_composite = composites['percentile']

    # Step 3: Enhance water clarity
    print("\nEnhancing water clarity...")
    final_composite = enhance_water_clarity(base_composite)

    print("\n✓ Optimized composite created")

    return final_composite, composites

def visualize_comparison(composites, aoi):
    """
    Create side-by-side comparison of different compositing methods
    """

    Map = geemap.Map()
    Map.centerObject(aoi, zoom=12)

    # Visualization parameters for coastal/reef
    water_vis = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 0.2,  # Lower max for water clarity
        'gamma': 1.5  # Higher gamma for shallow water detail
    }

    # Add different composites
    Map.addLayer(composites['percentile'], water_vis, 'Percentile', False)

    # Add enhanced indices for reef analysis
    # Reef index (shows coral/substrate)
    reef_index = composites['percentile'].normalizedDifference(['B3', 'B4'])
    reef_vis = {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'yellow', 'cyan', 'blue']}
    Map.addLayer(reef_index, reef_vis, 'Reef Index', False)

    # Water depth proxy
    depth_proxy = composites['percentile'].select('B2').divide(composites['percentile'].select('B3'))
    depth_vis = {'min': 0.5, 'max': 2, 'palette': ['darkblue', 'blue', 'cyan', 'yellow']}
    Map.addLayer(depth_proxy, depth_vis, 'Depth Proxy (B2/B3)', False)

    Map.addLayer(ee.Image().paint(aoi, 0, 2), {'palette': 'yellow'}, 'AOI')

    return Map

# =============================
# COMPLETE WORKFLOW
# =============================

def complete_reef_composite_workflow(aoi, start_date, end_date):
    """
    Complete workflow for creating best reef/coastal composite

    Args:
        aoi: Area of interest
        start_date: Start date
        end_date: End date
        mask_to_reef: Optional mask function for reef area
    """

    # Create optimized composite
    final_composite, all_composites = create_best_composite(
        aoi, start_date, end_date, max_cloud_cover=15
    )


    # Get projection info
    projection = final_composite.projection()
    crs = projection.crs().getInfo()
    scale = projection.nominalScale().getInfo()

    print(f"\n{'='*70}")
    print("FINAL COMPOSITE READY")
    print(f"{'='*70}")
    print(f"Projection: {crs}")
    print(f"Scale: {scale}m")
    print(f"Bands: {final_composite.bandNames().getInfo()}")

    # Create comparison map
    Map = visualize_comparison(all_composites, aoi)

    # Add final enhanced version
    final_vis = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 0.2,
        'gamma': 1.5
    }
    Map.addLayer(final_composite, final_vis, '★ FINAL ENHANCED', True)

    print("\n✓ Visualization ready")
    print("\nRECOMMENDATIONS:")
    print("  - Use 'FINAL ENHANCED' layer for best results")
    print("  - Adjust gamma (1.2-2.0) for shallow water detail")
    print("  - Lower max value (0.15-0.25) for clearer water")
    print("  - Check 'Reef Index' for substrate classification")
    print(f"{'='*70}\n")

    return final_composite, Map

In [6]:
sentinel_composite, Map = complete_reef_composite_workflow(
    aoi=aoi,
    start_date=start_date,
    end_date=end_date
)

# Display map
Map


CREATING OPTIMIZED COMPOSITE FOR COASTAL/REEF AREAS
Date range: 2024-01-01 to 2024-05-31
Max cloud cover: 15%

Creating Improved Sentinel-2 Composite
Initial images: 1027

Enhancing water clarity...

✓ Optimized composite created

FINAL COMPOSITE READY
Projection: EPSG:4326
Scale: 111319.49079327357m
Bands: ['B1', 'B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

✓ Visualization ready

RECOMMENDATIONS:
  - Use 'FINAL ENHANCED' layer for best results
  - Adjust gamma (1.2-2.0) for shallow water detail
  - Lower max value (0.15-0.25) for clearer water
  - Check 'Reef Index' for substrate classification



Map(center=[10.986191804458493, 122.0000000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
gpkg_dir = '/content/drive/MyDrive/Bathymetry/ECHOSOUNDER/2025'
output_dir = '/content/drive/MyDrive/Bathymetry/ECHOSOUNDER/bathymetry'

In [13]:
# =============================
# FAST SENTINEL-2 EXTRACTION
# =============================

def extract_reflectance_batch(image, points_fc, bands, scale=10):
    """
    Extract reflectance for multiple points in one server-side operation
    MUCH faster than individual point queries

    Args:
        image: ee.Image (Sentinel-2 composite)
        points_fc: ee.FeatureCollection (points with properties)
        bands: List of band names
        scale: Resolution in meters

    Returns:
        List of dictionaries with reflectance values
    """

    # Sample all points at once (SERVER-SIDE)
    sampled = image.select(bands).sampleRegions(
        collection=points_fc,
        scale=scale,
        geometries=False,  # Don't return geometries (faster)
        tileScale=4  # Use for large areas
    )

    # Get results as list
    results = sampled.getInfo()['features']

    # Extract properties
    extracted_data = []
    for feature in results:
        props = feature['properties']
        extracted_data.append(props)

    return extracted_data


def gpkg_to_ee_featurecollection(gdf):
    """
    Convert GeoDataFrame to Earth Engine FeatureCollection efficiently
    """
    features = []

    for idx, row in gdf.iterrows():
        # Create point geometry
        point = ee.Geometry.Point([row['lon'], row['lat']])

        # Create feature with properties
        feature = ee.Feature(point, {
            'id': int(idx),
            'lat': float(row['lat']),
            'lon': float(row['lon']),
            'depth': float(row['Depth']) if pd.notna(row['Depth']) else None,
            'wtemp': float(row['wtemp']) if pd.notna(row['wtemp']) else None
        })
        features.append(feature)

    return ee.FeatureCollection(features)


def process_gpkg_fast(file_path, sentinel_composite, bands, output_dir, chunk_size=1000):
    """
    Process GPKG file with optimized batching

    Args:
        file_path: Path to GPKG file
        sentinel_composite: ee.Image
        bands: List of bands to extract
        output_dir: Output directory
        chunk_size: Number of points per batch (adjust based on memory)

    Returns:
        DataFrame with results
    """

    file_name = os.path.basename(file_path)
    print(f"\n{'='*60}")
    print(f"Processing: {file_name}")
    print(f"{'='*60}")

    # Read GPKG
    gdf = gpd.read_file(file_path)
    total_points = len(gdf)
    print(f"Total points: {total_points}")

    # Process in chunks to avoid memory issues
    all_results = []

    for i in range(0, total_points, chunk_size):
        chunk_end = min(i + chunk_size, total_points)
        chunk_gdf = gdf.iloc[i:chunk_end]

        print(f"\nProcessing chunk {i//chunk_size + 1}/{(total_points-1)//chunk_size + 1} "
              f"(points {i} to {chunk_end})")

        try:
            # Convert to FeatureCollection
            points_fc = gpkg_to_ee_featurecollection(chunk_gdf)

            # Extract reflectance (single server call for entire chunk!)
            start_time = time.time()
            results = extract_reflectance_batch(
                sentinel_composite,
                points_fc,
                bands,
                scale=10
            )
            elapsed = time.time() - start_time

            print(f"  ✓ Extracted {len(results)} points in {elapsed:.2f}s "
                  f"({len(results)/elapsed:.1f} points/sec)")

            all_results.extend(results)

        except Exception as e:
            print(f"  ✗ Error in chunk: {e}")
            continue

    # Convert to DataFrame
    if all_results:
        df = pd.DataFrame(all_results)

        # Add source file
        df['source_file'] = file_name

        # Reorder columns
        cols = ['id', 'lat', 'lon', 'depth', 'wtemp'] + bands + ['source_file']
        df = df[[c for c in cols if c in df.columns]]

        # Remove rows with missing reflectance
        valid_df = df.dropna(subset=bands)

        print(f"\n{'='*60}")
        print(f"Results Summary:")
        print(f"  Total points: {total_points}")
        print(f"  Valid reflectance: {len(valid_df)}")
        print(f"  Success rate: {len(valid_df)/total_points*100:.1f}%")
        print(f"{'='*60}")

        return valid_df
    else:
        print("✗ No valid results")
        return None


def process_all_gpkg_fast(gpkg_dir, sentinel_composite, bands, output_dir,
                          chunk_size=1000, parallel=False):
    """
    Process all GPKG files in directory

    Args:
        gpkg_dir: Directory containing GPKG files
        sentinel_composite: ee.Image
        bands: List of bands
        output_dir: Output directory
        chunk_size: Points per batch
        parallel: Use parallel processing for multiple files (experimental)
    """

    os.makedirs(output_dir, exist_ok=True)

    # Find all GPKG files
    gpkg_files = [f for f in os.listdir(gpkg_dir) if f.endswith('.gpkg')]

    print(f"\n{'='*70}")
    print(f"FAST REFLECTANCE EXTRACTION")
    print(f"{'='*70}")
    print(f"Found {len(gpkg_files)} GPKG files")
    print(f"Bands: {bands}")
    print(f"Chunk size: {chunk_size} points/batch")
    print(f"{'='*70}")

    results_summary = []

    for file in gpkg_files:
        file_path = os.path.join(gpkg_dir, file)

        # Process file
        df = process_gpkg_fast(
            file_path,
            sentinel_composite,
            bands,
            output_dir,
            chunk_size
        )

        if df is not None and len(df) > 0:
            # Save to CSV
            out_csv = os.path.join(output_dir, f"reflectance_{file.replace('.gpkg', '')}.csv")
            df.to_csv(out_csv, index=False)
            print(f"\n✅ Saved: {out_csv}")

            results_summary.append({
                'file': file,
                'total_points': len(df) + df['id'].isna().sum(),
                'valid_points': len(df),
                'output': out_csv
            })
        else:
            print(f"\n❌ No valid reflectance in {file}")
            results_summary.append({
                'file': file,
                'total_points': 0,
                'valid_points': 0,
                'output': None
            })

    # Print final summary
    print(f"\n{'='*70}")
    print(f"PROCESSING COMPLETE")
    print(f"{'='*70}")

    summary_df = pd.DataFrame(results_summary)
    print(summary_df.to_string(index=False))

    total_valid = summary_df['valid_points'].sum()
    total_all = summary_df['total_points'].sum()
    print(f"\nTotal: {total_valid}/{total_all} points ({total_valid/total_all*100:.1f}% success)")
    print(f"{'='*70}\n")

    return summary_df


# =============================
# METHOD 2: BATCH EXPORT TO GOOGLE DRIVE FOR MULTIPLE FILES
# =============================

def export_multiple_gpkg_to_drive(gpkg_dir, sentinel_composite, bands, scale=10):
    """
    Export reflectance for all GPKG files to Google Drive in batch
    Fastest method - processes everything server-side

    Args:
        gpkg_dir: Directory with GPKG files
        sentinel_composite: ee.Image
        bands: List of bands to extract
        scale: Resolution in meters

    Returns:
        List of export tasks
    """

    print(f"\n{'='*70}")
    print("BATCH EXPORT TO GOOGLE DRIVE")
    print(f"{'='*70}")

    # Find all GPKG files
    gpkg_files = [f for f in os.listdir(gpkg_dir) if f.endswith('.gpkg')]
    print(f"Found {len(gpkg_files)} GPKG files")
    print(f"Bands to extract: {bands}")

    tasks = []

    for file in gpkg_files:
        file_path = os.path.join(gpkg_dir, file)
        file_name = file.replace('.gpkg', '')

        print(f"\n{'─'*60}")
        print(f"Processing: {file}")

        try:
            # Read GPKG
            gdf = gpd.read_file(file_path)
            n_points = len(gdf)
            print(f"  Points: {n_points}")

            # Convert to FeatureCollection
            print(f"  Converting to Earth Engine format...")
            points_fc = gpkg_to_ee_featurecollection(gdf)

            # Sample regions
            print(f"  Preparing export...")
            sampled = sentinel_composite.select(bands).sampleRegions(
                collection=points_fc,
                scale=30,
                geometries=False,
                tileScale=16  # Higher for large areas
            )

            # Create export task
            export_name = f'reflectance_{file_name}'
            task = ee.batch.Export.table.toDrive(
                collection=sampled,
                description=export_name,
                fileNamePrefix=export_name,
                fileFormat='CSV',
                folder='Bathymetry',
                selectors=['id', 'lat', 'lon', 'depth', 'wtemp'] + bands
            )

            # Start task
            task.start()
            tasks.append({
                'file': file,
                'task': task,
                'export_name': export_name,
                'points': n_points
            })

            print(f"  ✓ Export task started: {export_name}")

        except Exception as e:
            print(f"  ✗ Error: {e}")
            continue

    # Print summary
    print(f"\n{'='*70}")
    print(f"EXPORT TASKS STARTED")
    print(f"{'='*70}")
    print(f"Total files: {len(gpkg_files)}")
    print(f"Successful tasks: {len(tasks)}")

    print(f"\n{'Task Summary':}")
    print(f"{'─'*70}")
    for t in tasks:
        print(f"  {t['file']:30s} → {t['export_name']:40s} ({t['points']:,} points)")

    print(f"\n{'Next Steps':}")
    print(f"{'─'*70}")
    print("1. Monitor tasks at: https://code.earthengine.google.com/tasks")
    print("2. Wait for all tasks to complete (usually 5-30 minutes)")
    print("3. Download CSV files from Google Drive")
    print("4. Files will be in Drive root or 'EarthEngine' folder")
    print(f"{'='*70}\n")

    return tasks


def monitor_export_tasks(tasks, check_interval=60):
    """
    Monitor export task progress

    Args:
        tasks: List of task dictionaries from export_multiple_gpkg_to_drive
        check_interval: Seconds between status checks
    """

    print(f"\n{'='*70}")
    print("MONITORING EXPORT TASKS")
    print(f"{'='*70}")
    print(f"Checking every {check_interval} seconds...")
    print("Press Ctrl+C to stop monitoring\n")

    try:
        while True:
            all_done = True
            status_summary = {'COMPLETED': 0, 'RUNNING': 0, 'FAILED': 0, 'PENDING': 0}

            print(f"\n{'Status Update':}")
            print(f"{'─'*70}")

            for t in tasks:
                task = t['task']
                status = task.status()
                state = status['state']

                status_summary[state] = status_summary.get(state, 0) + 1

                if state != 'COMPLETED':
                    all_done = False

                # Status indicators
                if state == 'COMPLETED':
                    icon = '✓'
                    color = ''
                elif state == 'RUNNING':
                    icon = '⟳'
                    color = ''
                elif state == 'FAILED':
                    icon = '✗'
                    color = ''
                else:
                    icon = '○'
                    color = ''

                print(f"  {icon} {t['export_name']:40s} | {state:10s}")

            print(f"\n{'Summary':}")
            print(f"{'─'*70}")
            for state, count in status_summary.items():
                if count > 0:
                    print(f"  {state:10s}: {count}")

            if all_done:
                print(f"\n{'='*70}")
                print("✓ ALL TASKS COMPLETED!")
                print(f"{'='*70}")
                print("Download your CSV files from Google Drive")
                break

            time.sleep(check_interval)

    except KeyboardInterrupt:
        print("\n\nMonitoring stopped. Tasks continue running on server.")
        print("Check status at: https://code.earthengine.google.com/tasks")


def download_and_organize_results(drive_dir, output_dir, tasks):
    """
    After tasks complete, organize downloaded files

    Args:
        drive_dir: Directory where you downloaded Drive files
        output_dir: Where to organize final CSVs
        tasks: Task list from export_multiple_gpkg_to_drive
    """

    os.makedirs(output_dir, exist_ok=True)

    print(f"\n{'='*70}")
    print("ORGANIZING DOWNLOADED FILES")
    print(f"{'='*70}")

    organized = 0

    for t in tasks:
        export_name = t['export_name']
        csv_name = f"{export_name}.csv"

        # Look for file in drive_dir
        source = os.path.join(drive_dir, csv_name)
        dest = os.path.join(output_dir, csv_name)

        if os.path.exists(source):
            # Copy to output directory
            import shutil
            shutil.copy2(source, dest)

            # Load and show summary
            df = pd.read_csv(dest)
            print(f"✓ {csv_name:40s} | {len(df):,} points")
            organized += 1
        else:
            print(f"✗ {csv_name:40s} | Not found in {drive_dir}")

    print(f"\n{'='*70}")
    print(f"Organized {organized}/{len(tasks)} files")
    print(f"Output directory: {output_dir}")
    print(f"{'='*70}\n")


# =============================
# CONVENIENCE FUNCTION: ALL-IN-ONE
# =============================

def process_multiple_gpkg_drive_export(gpkg_dir, sentinel_composite, bands,
                                      monitor=True, check_interval=60):
    """
    All-in-one function: Export all GPKG files and optionally monitor

    Args:
        gpkg_dir: Directory with GPKG files
        sentinel_composite: ee.Image
        bands: List of bands
        monitor: Whether to monitor task progress
        check_interval: Seconds between monitoring checks

    Returns:
        List of export tasks
    """

    # Start all exports
    tasks = export_multiple_gpkg_to_drive(
        gpkg_dir=gpkg_dir,
        sentinel_composite=sentinel_composite,
        bands=bands,
        scale=10
    )

    # Monitor if requested
    if monitor and len(tasks) > 0:
        print("\nStarting task monitoring in 10 seconds...")
        time.sleep(10)  # Give tasks time to start
        monitor_export_tasks(tasks, check_interval)

    return tasks


# =============================
# USAGE EXAMPLES
# =============================

if __name__ == "__main__":
    bands = ['B1', 'B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
    # Start the export process
    tasks = export_multiple_gpkg_to_drive(
        gpkg_dir=gpkg_dir,
        sentinel_composite=sentinel_composite,
        bands=bands,
        scale=10
    )

    if tasks:
        print(f"\n{'✅ EXPORTS STARTED ':*^70}")
        print(f"Monitoring tasks at: https://code.earthengine.google.com/tasks")

        # Optional: Monitor progress
        monitor_export_tasks(tasks, check_interval=30)
    else:
        print("❌ No tasks started. Check your GPKG directory path.")

print("\n" + "="*70)
print("OPTIMIZATION GUIDE")
print("="*70)
print("Original method: ~1-2 points/sec")
print("Batch method:    ~50-200 points/sec (10-100x faster)")
print("Export method:   ~1000+ points/sec (500x faster)")
print("\nFor 1.4 MB (~10k points):")
print("  Original: 1-3 hours")
print("  Batch:    1-3 minutes ⚡")
print("  Export:   5-15 seconds ⚡⚡⚡")
print("="*70)


BATCH EXPORT TO GOOGLE DRIVE
Found 4 GPKG files
Bands to extract: ['B1', 'B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

────────────────────────────────────────────────────────────
Processing: bathymetry_calatagantrack1_dry2025.gpkg
  Points: 6487
  Converting to Earth Engine format...
  Preparing export...
  ✓ Export task started: reflectance_bathymetry_calatagantrack1_dry2025

────────────────────────────────────────────────────────────
Processing: bathymetry_calatagantrack2_dry2025.gpkg
  Points: 24235
  Converting to Earth Engine format...
  Preparing export...
  ✓ Export task started: reflectance_bathymetry_calatagantrack2_dry2025

────────────────────────────────────────────────────────────
Processing: bathymetry_calatagantrack3_dry2025.gpkg
  Points: 5796
  Converting to Earth Engine format...
  Preparing export...
  ✓ Export task started: reflectance_bathymetry_calatagantrack3_dry2025

────────────────────────────────────────────────────────────
Processing: bathymetry_calatagan